In [1]:
!pip install git+https://github.com/huggingface/transformers
!pip install sentencepiece
!pip install git+https://github.com/huggingface/accelerate
!pip install datasets
!pip install gdown

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-yf7k9b6l
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-yf7k9b6l
  Resolved https://github.com/huggingface/transformers to commit 94056b57beb4499f4f74d5d88a41e8266cc01778
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.29.0.dev0-py3-none-any.whl size=7012505 sha256=e01057b9d80fc60667a945a23669df16f05a133aec36ef97a84510a04f782a19
  Stored in directory: /tmp/pip-ephem-wheel-cache-f31odnek/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.28.1
    Uninstalling transformers-4.28.1:
      Successfully uninstalled transformers-4.28.1
  Cloning https://github.com/huggingface

In [2]:
import multiprocessing
cpu_cores = multiprocessing.cpu_count()
cpu_cores

2

In [3]:
import json
import os
from typing import Union,List
import sys

import torch
from transformers import XGLMTokenizer, XGLMForCausalLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq
from datasets import load_dataset


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
print('load_model')

load_model


In [5]:
def load_model(base_model:str="facebook/xglm-564M"):
    tokenizer = XGLMTokenizer.from_pretrained(base_model)
    model = XGLMForCausalLM.from_pretrained(base_model,
                                           torch_dtype=torch.float16)
    new_tokens = ['<human>:', '<bot>:']
    tokenizer.add_tokens(list(new_tokens))
    model.resize_token_embeddings(len(tokenizer))
    
    return model,tokenizer
model,tokenizer = load_model()

In [6]:
import gdown

url = 'https://drive.google.com/uc?export=download&id=1jbbUtwgwoSQgGnXxzTh-nMReVzEU7ZTU&confirm=t&uuid=d79e2e78-51de-466f-9ceb-3944606141a2&at=AKKF8vwcgi95TGSnSQUNCKx4NTqS:1682865249145'
output = 'output.jsonl'
gdown.download(url, output, quiet=False)


Downloading...
From: https://drive.google.com/uc?export=download&id=1jbbUtwgwoSQgGnXxzTh-nMReVzEU7ZTU&confirm=t&uuid=d79e2e78-51de-466f-9ceb-3944606141a2&at=AKKF8vwcgi95TGSnSQUNCKx4NTqS:1682865249145
To: /kaggle/working/output.jsonl
100%|██████████| 167M/167M [00:01<00:00, 132MB/s]  


'output.jsonl'

In [7]:
from datasets import load_dataset
datasets = load_dataset('json',data_files='output.jsonl')
datasets

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-07633b6dc18ebeb6/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['Background:', '<human>:', '<bot>:'],
        num_rows: 116288
    })
})

In [8]:
def format_prompt(prompt):
    return {'prompt':f"{prompt['Background:']} : {prompt['<human>:']} : {prompt['<bot>:']}"}
# format data like  context ......
def preprocess(prompt):
    data = tokenizer(
        prompt['prompt'],
        truncation=True,
        max_length=256,
        padding=False,
        return_tensors=None,
    )
    data['input_ids'].append(tokenizer.eos_token_id)
    data['attention_mask'].append(1)
    data['labels'] = data['input_ids'].copy()
    return data

In [9]:
print('preprocess')

preprocess


In [10]:

datasets = datasets['train']
datasets = datasets.map(format_prompt,remove_columns=['Background:', '<human>:', '<bot>:'],num_proc=cpu_cores)
datasets = datasets.map(lambda x:{'token':len(tokenizer.tokenize(x['prompt']))},num_proc=cpu_cores)
datasets = datasets.filter(lambda x:x['token']<255,num_proc=cpu_cores)
datasets = datasets.map(preprocess,remove_columns=['prompt','token'],num_proc=cpu_cores) 
datasets = datasets.filter(lambda x:x['input_ids'][0] == 2,num_proc=cpu_cores)
datasets = datasets.filter(lambda x:x['input_ids'][-1] == 2,num_proc=cpu_cores)  

datasets

#1:   0%|          | 0/58144 [00:00<?, ?ex/s]

#0:   0%|          | 0/58144 [00:00<?, ?ex/s]

#1:   0%|          | 0/58144 [00:00<?, ?ex/s]

#0:   0%|          | 0/58144 [00:00<?, ?ex/s]

#0:   0%|          | 0/59 [00:00<?, ?ba/s]

...

#1:   0%|          | 0/31 [00:00<?, ?ba/s]

#0:   0%|          | 0/31 [00:00<?, ?ba/s]

#0:   0%|          | 0/31 [00:00<?, ?ba/s]

#1:   0%|          | 0/31 [00:00<?, ?ba/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 60916
})

In [11]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(
            tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True 
        )

In [85]:
from accelerate import Accelerator
from torch.utils.data import DataLoader
from transformers import AdamW,get_scheduler
from torch.cuda.amp import  GradScaler
import math
def train(model):
    accelerator = Accelerator()

    batch_size = 256
    mini_batch_size = 8
    gradient_accumulation_steps = batch_size //mini_batch_size
    train_dataloader = DataLoader(
        datasets, shuffle=True, batch_size=mini_batch_size, collate_fn=data_collator
    )

    num_epochs = 3
    l_data_loader = len(train_dataloader)
    num_training_steps = num_epochs * l_data_loader

    optimizer = AdamW(model.parameters(), lr=3e-7)
    lr_scheduler = get_scheduler(
      "linear",
      optimizer=optimizer,
      num_warmup_steps=100,
      num_training_steps=math.ceil(num_training_steps/gradient_accumulation_steps),
    )
    accelerator.print(num_training_steps)

    model, optimizer, train_dataloader, lr_scheduler = accelerator.prepare(model, optimizer, train_dataloader, lr_scheduler)
    output_dir_min_loss = 'xglm-checkpoint-min-loss'
    OPTIMIZER_NAME = "optimizer.pt"
    SCHEDULER_NAME = "scheduler.pt"

    scaler = GradScaler()
    accelerator.save_state(output_dir="my_checkpoint")
    accelerator.print('start')
    for epoch in range(num_epochs):
        for index,data in enumerate(train_dataloader):
            with torch.amp.autocast(device_type='cuda', dtype=torch.float16):
                outputs = model(**data)
                loss = outputs.loss / gradient_accumulation_steps
                accelerator.backward(scaler.scale(loss))
            if (index + 1) % gradient_accumulation_steps == 0 or (index+1) == math.ceil(l_data_loader/torch.cuda.device_count()) or (index+1) == math.floor(l_data_loader/torch.cuda.device_count()):
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()
                if (index+1) % gradient_accumulation_steps**2 == 0:
                    accelerator.print(epoch,index+1,loss)
                elif (index+1) == math.ceil(l_data_loader/torch.cuda.device_count()) or (index+1) == math.floor(l_data_loader/torch.cuda.device_count()):
                    accelerator.print(epoch,index+1,loss)
        accelerator.wait_for_everyone() 


In [86]:
from accelerate import notebook_launcher
print('train')
notebook_launcher(train, args=(model,),num_processes=torch.cuda.device_count()) 

train
Launching training on 2 GPUs.


/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


22845
start
0 1024 0
0 2048 0
0 3072 0
0 3807 31
0 3808 0
epoch 0
7615 3807
1 1024 0
1 2048 0
1 3072 0
1 3807 31
1 3808 0
epoch 1
7615 3807
2 1024 0
2 2048 0
2 3072 0
2 3807 31
2 3808 0
epoch 2
7615 3807
